In [1]:
# 모듈 추가
import requests
from urllib.parse import urlparse
import pandas as pd

In [2]:
lon = 127.947727443701; lat = 37.337061376149 # 국립공원연구원 위치
page=1 # 첫번째 페이지
url = "https://dapi.kakao.com/v2/local/search/category.json?&category_group_code=CS2&x="\
+str(lon)+"&y="+str(lat)+"&page="+str(page)+"&radius=1000" # 카카오맵 API
json_obj = requests.get(urlparse(url).geturl(),headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"}).json()

In [3]:
# 첫번째 편의점 정보
json_obj['documents'][0]

{'address_name': '강원 원주시 개운동 322',
 'category_group_code': 'CS2',
 'category_group_name': '편의점',
 'category_name': '가정,생활 > 편의점 > 미니스톱',
 'distance': '320',
 'id': '11230024',
 'phone': '1577-9621',
 'place_name': '미니스톱 원주현대점',
 'place_url': 'http://place.map.kakao.com/11230024',
 'road_address_name': '강원 원주시 남원로612번길 9',
 'x': '127.951338709114',
 'y': '37.3370900547245'}

In [4]:
# 첫번째 편의점 정보 데이터프레임
pd.DataFrame(json_obj['documents'][0], index=[0])[['place_name','road_address_name', 'distance', 'x', 'y']]

,place_name,road_address_name,distance,x,y
0,미니스톱 원주현대점,강원 원주시 남원로612번길 9,320,127.951338709114,37.3370900547245


In [5]:
# 반복문을 위한 데이터프레임 구성
df = pd.DataFrame(columns = ['place_name','road_address_name', 'distance', 'x', 'y'])
df_s = pd.DataFrame(json_obj['documents'][0], index=[0])[['place_name','road_address_name', 'distance', 'x', 'y']]
df = df.append(df_s)
df.head()

,place_name,road_address_name,distance,x,y
0,미니스톱 원주현대점,강원 원주시 남원로612번길 9,320,127.951338709114,37.3370900547245


In [6]:
# 메타정보
json_obj['meta']

{'is_end': False, 'pageable_count': 35, 'same_name': None, 'total_count': 35}

In [7]:
# 메타정보 조건문 정의
if json_obj['meta']['is_end'] == False:
    print("False")
else:
    print("True")

False


In [8]:
# 편의점 검색 함수 정의
def search_CVS(lon, lat):
    df = pd.DataFrame(columns = ['place_name','road_address_name', 'distance', 'x', 'y'])
    page = 1
    while True:
        url = "https://dapi.kakao.com/v2/local/search/category.json?&category_group_code=CS2&x="\
        +str(lon)+"&y="+str(lat)+"&page="+str(page)+"&radius=1000"
        json_obj = requests.get(urlparse(url).geturl(),headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"}).json()
        for document in json_obj['documents']:
            df_s = pd.DataFrame(document, index=[0])[['place_name','road_address_name', 'distance', 'x', 'y']]
            df = df.append(df_s)
        if json_obj['meta']['is_end'] == False:
            page += 1
        else:
            return df

In [9]:
# 편의점 검색 함수 테스트
df = search_CVS(127.947727443701, 37.337061376149)

In [10]:
# 편의점 정보
df.head()

,place_name,road_address_name,distance,x,y
0,미니스톱 원주현대점,강원 원주시 남원로612번길 9,320,127.951338709114,37.3370900547245
0,GS25 원주원동점,강원 원주시 남산로 15,496,127.944191621538,37.3405350658268
0,CU 원주향교점,강원 원주시 남원로 611-1,310,127.95105491442797,37.33794017550028
0,CU 원주명륜점,강원 원주시 예술관길 28,687,127.941845341032,37.3330178172923
0,CU 원주개운주공점,강원 원주시 남원로606번길 16,451,127.952817527098,37.3369592216809


In [11]:
# 가장 가까운 편의점 확인
df.sort_values(by=['distance'], ascending=True).head()

,place_name,road_address_name,distance,x,y
0,CU 원주향교점,강원 원주시 남원로 611-1,310,127.95105491442797,37.33794017550028
0,세븐일레븐 원주남원로점,강원 원주시 남원로 611-1,310,127.951056076815,37.3379428691488
0,미니스톱 원주현대점,강원 원주시 남원로612번길 9,320,127.951338709114,37.3370900547245
0,GS25 원주남원로점,강원 원주시 남원로 629,431,127.951834047153,37.3391439490561
0,CU 원주개운주공점,강원 원주시 남원로606번길 16,451,127.952817527098,37.3369592216809


In [12]:
# 편의점 개수
len(df)

35

In [13]:
# 경도
df.iloc[0, 3]

'127.951338709114'

In [14]:
# 위도
df.iloc[0, 4]

'37.3370900547245'

In [15]:
# QGIS 파이썬콘솔: 국립공원연구원 임시 레이어 추가
'''
vl = QgsVectorLayer("Point?crs=EPSG:4326", "국립공원연구원", "memory")
pr = vl.dataProvider()
pr.addAttributes([QgsField("place_name", QVariant.String),
                  QgsField("lon", QVariant.Double),
                  QgsField("lat", QVariant.Double)])
vl.updateFields()
for i in range(len(df)):
    f = QgsFeature()
    f.setGeometry(QgsGeometry.fromPointXY(127.947727443701, 37.337061376149)))
    f.setAttributes(['국립공원연구원', 127.947727443701, 37.337061376149])
    pr.addFeature(f)
vl.updateExtents()
QgsProject.instance().addMapLayer(vl)
'''

'\nvl = QgsVectorLayer("Point?crs=EPSG:4326", "국립공원연구원", "memory")\npr = vl.dataProvider()\npr.addAttributes([QgsField("place_name", QVariant.String),\n                  QgsField("lon", QVariant.Double),\n                  QgsField("lat", QVariant.Double)])\nvl.updateFields()\nfor i in range(len(df)):\n    f = QgsFeature()\n    f.setGeometry(QgsGeometry.fromPointXY(127.947727443701, 37.337061376149)))\n    f.setAttributes([\'국립공원연구원\', 127.947727443701, 37.337061376149])\n    pr.addFeature(f)\nvl.updateExtents()\nQgsProject.instance().addMapLayer(vl)\n'

In [16]:
# QGIS 파이썬콘솔: 편의점 검색 함수 정의
def search_CVS(lon, lat):
    df = pd.DataFrame(columns = ['place_name','road_address_name', 'distance', 'x', 'y'])
    page = 1
    while True:
        url = "https://dapi.kakao.com/v2/local/search/category.json?&category_group_code=CS2&x="\
        +str(lon)+"&y="+str(lat)+"&page="+str(page)+"&radius=1000"
        json_obj = requests.get(urlparse(url).geturl(),headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"}).json()
        for document in json_obj['documents']:
            df_s = pd.DataFrame(document, index=[0])[['place_name','road_address_name', 'distance', 'x', 'y']]
            df = df.append(df_s)
        if json_obj['meta']['is_end'] == False:
            page += 1
        else:
            vl = QgsVectorLayer("Point?crs=EPSG:4326", "CVS", "memory")
            pr = vl.dataProvider()
            pr.addAttributes([QgsField("place_name", QVariant.String),
                              QgsField("road_address_name", QVariant.String),
                              QgsField("distance",  QVariant.Int),
                              QgsField("lon", QVariant.Double),
                              QgsField("lat", QVariant.Double)])
            vl.updateFields()
            for i in range(len(df)):
                f = QgsFeature()
                f.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(float(df.iloc[i, 4]), float(df.iloc[i, 3]))))
                f.setAttributes([df.iloc[i, 0], df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 4], df.iloc[i, 3]])
                pr.addFeature(f)
            vl.updateExtents()
            QgsProject.instance().addMapLayer(vl)
            break

In [17]:
# 결과 확인
# search_CVS(127.947727443701, 37.337061376149)